In [1]:
import re
import os


sparql_keywords = {
    'SELECT', 'CONSTRUCT', 'ASK', 'DESCRIBE', 'BIND', 'WHERE', 'LIMIT',
    'VALUES', 'DISTINCT', 'AS', 'FILTER', 'ORDER', 'BY', 'SERVICE', 'OFFSET',
    'NOT', 'EXISTS', 'OPTIONAL', 'UNION', 'FROM', 'GRAPH', 'NAMED', 'DESC',
    'ASC', 'REDUCED', 'STR', 'LANG', 'LANGMATCHES', 'REGEX', 'BOUND', 'DATATYPE',
    'ISBLANK', 'ISLITERAL', 'ISIRI', 'ISURI', 'GROUP_CONCAT', 'GROUP', 'DELETE', 'CLEAR',
    'CREATE', 'COPY', 'DROP', 'INSERT', 'LOAD', 'DATA', 'INTO', 'WITH', 'ALL', 'SILENT',
    'DEFAULT', 'USING', 'MD5', 'SHA1', 'SHA256', 'SHA384', 'SHA512', 'STRSTARTS',
    'STRENDS', 'SAMETERM', 'ISNUMERIC', 'UCASE', 'SUBSTR', 'STRLEN', 'STRBEFORE', 'STRAFTER',
    'REPLACE', 'LEVENSHTEIN_DIST', 'LCASE', 'ENCODE_FOR_URI', 'CONTAINS', 'CONCAT',
    'COALESCE', 'CHOOSE_BY_MAX', 'CHOOSE_BY_MIN', 'YEAR', 'DAY', 'TZ', 'TIMEZONE', 'HOURS',
    'MINUTES', 'MONTH', 'NOW', 'DUR_TO_USECS', 'SECONDS_DBL', 'USECS_TO_DUR', 'IF', 'MINUS',
    'AVG', 'COUNT', 'MAX', 'MIN', 'SAMPLE', 'SUM', 'ABS', 'ADD', 'BASE', 'CEIL', 'COS', 'FLOOR',
    'HAMMING_DIST', 'HAVERSINE_DIST', 'LN', 'LOG2', 'MOD', 'POWER', 'RADIANS', 'RAND',
    'ROUND', 'ROUNDDOWN', 'ROUNDUP', 'TAN', 'VAR', 'VARP'
}

In [ ]:
from generator_utils import encode
os.mkdir('LC_QUAD')
files = ['train.sparql', 'test.sparql', 'dev.sparql']
for filename in files:
    with open(f'LC_QUAD/{filename}', 'w', encoding='utf-8') as out:
        with open(f'{filename}', 'r', encoding='utf-8') as file:
            cnt = 0
            for line in file:
                cnt += 1
                normalize_s = []
                sparql_query = line.replace(' math_gt', '>')
                for token in sparql_query.split():
                    beginning_subtoken = re.search(r'^\w+', token)
                    if beginning_subtoken is not None:
                        beginning_subtoken = beginning_subtoken.group()
                        if beginning_subtoken.upper() in sparql_keywords:
                            token = re.sub(r'^\w+', beginning_subtoken.lower(), token)
                    normalize_s.append(token)
                sparql_query = " ".join(normalize_s)
                out.write(sparql_query + '\n')

In [3]:
from generator_utils import decode, encode
prefixes = {'dbo:', 'dbp:', 'dbr:', 'dbc:', 'dct:', 'rdf:', 'rdfs:', 'xsd:', 'geo:', 'foaf:', 'yago:', 'owl:', 'skos:',
           'bif:', 'dc:', 'freebase:', 'georss:', 'rss:', 'schema:', 'vrank:', 'geof:'}

for file_set in ['dev', 'test', 'train']:
    with open(f'{file_set}.sparql', 'w', encoding='utf-8') as out_file:
        with open(f'../{file_set}.sparql', 'r', encoding='utf-8') as in_file:
            for line in in_file:
                sparql_query = decode(line.strip())
                normalize_s = []
                prev_open = False
                for token in sparql_query.split():
                    pr = token[:token.find(':')+1]
                    if pr in prefixes:
                        token = '<' + token + '>'
                    beginning_subtoken = re.search(r'^\w+', token)
                    if beginning_subtoken is not None:
                        beginning_subtoken = beginning_subtoken.group()
                        if beginning_subtoken.upper() in sparql_keywords:
                            token = re.sub(r'^\w+', beginning_subtoken.lower(), token)
                    normalize_s.append(token)
                sparql_query = " ".join(normalize_s)
                sparql_query = encode(sparql_query)
                sparql_query = ' '.join(sparql_query.strip().split())
                sparql_query = re.sub(r'(_>\spar_open(((?!_>\spar_open).)*?)par_close)', r'_ attr_open \2 attr_close >', sparql_query)
                out_file.write(sparql_query + '\n')
                

In [6]:
for file_set in ['dev', 'test', 'train']:
    with open(f'{file_set}.sparql', 'w', encoding='utf-8') as out_file:
        with open(f'../{file_set}.sparql', 'r', encoding='utf-8') as in_file:
            for line in in_file:
                sparql_query = ' '.join(line.strip().split())
                out_file.write(sparql_query + '\n')

In [21]:
with open('../tmp/dev.output', 'r', encoding='utf-8') as in_file:
    uniq = set()
    for line in in_file:
        print(line)
        line = re.sub(r' ?([!"#$%&\'(’)*+,-./:;=?@\\^_`{|}~]) ?', r'\1', line)
        line = line.replace('attr_close>', 'attr_close >').replace('_attr_open', '_ attr_open')
        line = line.replace(' [ ', ' [').replace(' ] ', '] ')
        line = line.replace('_obd_', ' _obd_ ').replace('_oba_', ' _oba_ ')
        line = line.replace('||', ' or_logical ')
        for t in re.findall(r'([^<>\s]+ )', line):
            uniq.add(t)
        print(line)

81	select var_a where brack_open <dbr_Mały_Powstaniec> <dbo_location> var_a brack_close

81	select var_a where brack_open <dbr_Mały_Powstaniec> <dbo_location> var_a brack_close

14	select var_a where brack_open <dbr_Avro_Anson_Memorial> <dbo_location> var_a brack_close

14	select var_a where brack_open <dbr_Avro_Anson_Memorial> <dbo_location> var_a brack_close

3	select var_a where brack_open <dbr_Raibania_fort> <dbo_designer> var_a brack_close

3	select var_a where brack_open <dbr_Raibania_fort> <dbo_designer> var_a brack_close

94	select var_a where brack_open <dbr_Tomb_of_Sher_Shah_Suri> <dbp_complete> var_a brack_close

94	select var_a where brack_open <dbr_Tomb_of_Sher_Shah_Suri> <dbp_complete> var_a brack_close

35	select var_a where brack_open <dbr_Haji_Gayib ’ s_bathhouse> <dbo_location> var_a brack_close

35	select var_a where brack_open <dbr_Haji_Gayib’s_bathhouse> <dbo_location> var_a brack_close

31	select var_a where brack_open <dbr_Civil_Rights_Memorial> <dct_subject> var

In [33]:
with open('../DBNQA_process/dev.sparql', 'r', encoding='utf-8') as in_file:
    uniq = set()
    for line in in_file:
        for t in re.findall(r'([^<>\s]+ )', line):
            if 'A' <= t[0] <= 'Z': continue
            if not t.endswith('_ '):
                uniq.add(t)

In [34]:
uniq

{'1652–1830 ',
 '2001–present ',
 'nautical ',
 '1998–2015 ',
 '1816 ',
 'painting ',
 '_Islands ',
 'novelist ',
 '0 ',
 '2001_TV_serial ',
 'footballer,_born_1970 ',
 '1778 ',
 'season_13 ',
 'social_network ',
 'materials_scientist ',
 '1968_film ',
 'aircraft ',
 '1990 ',
 '1804 ',
 '1999–2000 ',
 '1578–1624 ',
 'bread_baking ',
 'Žďár_nad_Sázavou_District ',
 'transport_company ',
 'physiologist ',
 'cornerback ',
 '-1877 ',
 '19th_century ',
 '1919 ',
 '415029 ',
 '1637–1640 ',
 '1964_film ',
 'revue ',
 'duo ',
 'government_official ',
 'philosophy ',
 'ninth_generation ',
 'forensic_anthropologist ',
 'instrumental ',
 'footballer,_born_1920 ',
 'graphic_designer ',
 'icon,_1130 ',
 '2011 ',
 '1738 ',
 'season_2 ',
 'singer-songwriter ',
 "var_name,'dbr_1050s' ",
 '2009_film ',
 'psychologist ',
 "men's_handball ",
 'wrestler ',
 '1929 ',
 '1993_album ',
 "var_name,'dbo_televisionEpisode' ",
 'ask ',
 'supermarket ',
 '1918–19 ',
 '1979-1989 ',
 'standard ',
 'animated_TV_serie